In [1]:
%load_ext nb_black
# !pip install nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import metrics as m
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE  # doctest: +NORMALIZE_WHITESPACE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
import xgboost as xgb
from sklearn.pipeline import make_pipeline
from mlxtend.feature_selection import ColumnSelector
from sklearn import model_selection
from mlxtend.classifier import StackingClassifier
from thundersvm import SVC as svmgpu

import warnings

warnings.filterwarnings("ignore")
randomseed = 7

Using TensorFlow backend.


<IPython.core.display.Javascript object>

# 1. Read the dataset

In [4]:
x_original = pd.read_csv("../dataset/_TargetLable_forischemic.txt")

conditions = [
    (x_original.L100800 < 100),
    (x_original.L100800 >= 100) & (x_original.L100800 < 126),
    (x_original.L100800 >= 126),
]
choices = [0, 1, 2]
x_original["CLASS"] = np.select(conditions, choices, default=0)
x_original = x_original[
    [
        "Unnamed: 0",
        "L100800",
        "L104600",
        "L103000",
        "S000300",
        "L101700",
        "L100700",
        "FIELD_33",
        "FIELD_38",
        "FIELD_40",
        "FIELD_31",
        "SEX",
        "AGE",  #'CLASS',
        "FIELD_16",
        "FIELD_23",
        "FIELD_15",
        "FIELD_22",
        "FIELD_17",
        "FIELD_24",
    ]
]

<IPython.core.display.Javascript object>

In [5]:
y_original = pd.read_csv("../dataset/_XLable_forischemic.txt")

conditions = [
    (y_original.L100800 < 100),
    (y_original.L100800 >= 100) & (y_original.L100800 < 126),
    (y_original.L100800 >= 126),
]

choices = [0, 1, 2]
y_original["CLASS"] = np.select(conditions, choices, default=0)

y_original = y_original[["Unnamed: 0", "CLASS"]]

<IPython.core.display.Javascript object>

In [6]:
data = pd.merge(
    x_original, y_original, how="inner", left_on="Unnamed: 0", right_on="Unnamed: 0"
)

<IPython.core.display.Javascript object>

In [7]:
# filter the data set
data = data[data.FIELD_16 != 1]  # exclude people who are diagnosed for (diabetes)
data = data[data.FIELD_23 != 1]  # exclude people who are on medication for diabetes

# data = data[data.FIELD_15 != 1]  # exclude people who are diagnosed for (high blood pressure)
# data = data[    data.FIELD_22 != 1]  # exclude people who are on medication for high blood pressure

# data = data[data.FIELD_17 != 1]  # exclude people who are diagnosed for hyperlipidemia
# data = data[    data.FIELD_24 != 1]  # exclude people who are on medication for hyperlipidemia

print(data.shape)

(186439, 20)


<IPython.core.display.Javascript object>

In [8]:
data = data[
    [
        "L100800",
        "L104600",
        "L103000",
        "S000300",
        "L101700",
        "L100700",
        "FIELD_33",
        "FIELD_38",
        "FIELD_40",
        "FIELD_31",
        "SEX",
        "AGE",
        "CLASS",
    ]
]
data = data.dropna()
print(data.shape)
data.head()

(66185, 13)


,L100800,L104600,L103000,S000300,L101700,L100700,FIELD_33,FIELD_38,FIELD_40,FIELD_31,SEX,AGE,CLASS
1,78.0,5.28,41.0,20.2,15.0,3.8,1.0,2.0,1.0,0.0,1.0,46.0,0
2,86.0,4.89,50.0,21.0,14.0,3.1,1.0,2.0,0.0,0.0,1.0,47.0,0
4,90.0,5.74,50.0,25.5,12.0,3.4,1.0,0.0,1.0,0.0,1.0,52.0,0
5,88.0,5.77,41.0,25.4,13.0,4.4,1.0,0.0,0.0,1.0,1.0,53.0,0
10,86.0,5.83,45.0,21.2,17.0,3.9,1.0,0.0,1.0,1.0,1.0,37.0,0


<IPython.core.display.Javascript object>

# 2. Downsample the majority class and upsample the minority

In [9]:
diabetic = data[data.CLASS == 2]
prediabetic = data[data.CLASS == 1]
normal = data[data.CLASS == 0]

print(diabetic.shape[0], prediabetic.shape[0], normal.shape[0])

1100 18750 46335


<IPython.core.display.Javascript object>

In [10]:
diabetic_test = diabetic.sample(200, random_state=randomseed)
prediabetic_test = prediabetic.sample(200, random_state=randomseed)
normal_test = normal.sample(200, random_state=randomseed)
test = pd.concat([diabetic_test, prediabetic_test, normal_test])

diabetic_train = diabetic.drop(diabetic_test.index)
prediabetic_train = prediabetic.drop(prediabetic_test.index)
# .sample(
#     10 * diabetic_train.shape[0], random_state=randomseed
# )
normal_train = normal.drop(normal_test.index).sample(
    prediabetic_train.shape[0]
    #     10 * diabetic_train.shape[0], random_state=randomseed
)
train = pd.concat([diabetic_train, diabetic_train, prediabetic_train, normal_train])

<IPython.core.display.Javascript object>

In [11]:
xtrain = train.iloc[:, :-1]
ytrain = train.iloc[:, -1]
xtest = test.iloc[:, :-1]
ytest = test.iloc[:, -1]

<IPython.core.display.Javascript object>

In [12]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
xtrain = scaler.fit_transform(xtrain)
xtest = scaler.transform(xtest)

<IPython.core.display.Javascript object>

In [13]:
from imblearn.over_sampling import SMOTE, SMOTENC  # doctest: +NORMALIZE_WHITESPACE

randomseed = 42

sm = SMOTENC(
    random_state=randomseed,
    categorical_features=[6, 7, 8, 9, 10],
    sampling_strategy="minority",
)
X_res, y_res = sm.fit_resample(xtrain, ytrain)

print("Resampled dataset shape %s" % Counter(y_res))
print(
    y_res[y_res == 0].shape[0], y_res[y_res == 1].shape[0], y_res[y_res == 2].shape[0]
)
print(X_res.shape, y_res.shape)

xtrain = X_res
ytrain = y_res

Resampled dataset shape Counter({2: 18550, 1: 18550, 0: 18550})
18550 18550 18550
(55650, 12) (55650,)


<IPython.core.display.Javascript object>

# 3. Generate the classifier models based on the selected 12 features

In [14]:
# 3.1. 12 Features

<IPython.core.display.Javascript object>

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

parameters = {
    "max_features": ("auto", "sqrt", "log2"),
    "n_estimators": [10, 50, 100, 150, 200, 300, 700],
    "max_depth": [2, 6, 8, 10, 12],
    "min_samples_split": [2, 6, 8, 10, 12],
    "min_samples_leaf": [2, 6, 8, 10, 12],
}


rf_clf = RandomForestClassifier(n_jobs=-1)
rf_clf = GridSearchCV(rf_clf, parameters)
rf_clf.fit(xtrain, ytrain)

In [ ]:
rf_clf.best_estimator_

In [ ]:
rf_12 = RandomForestClassifier(
    random_state=randomseed,
    n_estimators=100,
    max_depth=12,
    min_samples_split=2,
    min_samples_leaf=10,
    max_features="auto",
)

In [ ]:
rf_12.fit(xtrain, ytrain)

In [30]:
from thundergbm import TGBMClassifier

clf = TGBMClassifier(bagging=1, n_parallel_trees=50, n_trees=500, depth=12)

<IPython.core.display.Javascript object>

In [31]:
clf.fit(xtrain, ytrain)

TGBMClassifier(bagging=1, column_sampling_rate=1.0, depth=12, gamma=1.0,
               lambda_tgbm=1.0, learning_rate=1.0, max_num_bin=255,
               min_child_weight=1.0, n_gpus=1, n_parallel_trees=50, n_trees=500,
               num_class=3, objective='multi:softmax', tree_method='auto',
               verbose=1)

<IPython.core.display.Javascript object>

In [32]:
ypred = clf.predict(xtest)

<IPython.core.display.Javascript object>

In [33]:
score = clf.score(xtest, ytest)
print(score)

0.595


<IPython.core.display.Javascript object>